# Exercice 1

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv(r"C:/Users/tzolz/OneDrive - De Vinci/ESILV/A4 ESILV/Python for DA/td8/boston_house_prices.csv", header=1)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [2]:
X = df.drop("MEDV", axis=1) 
y = df["MEDV"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled)
print(X_test_scaled)

[[ 1.28770177 -0.50032012  1.03323679 ...  0.84534281 -0.07433689
   1.75350503]
 [-0.33638447 -0.50032012 -0.41315956 ...  1.20474139  0.4301838
  -0.5614742 ]
 [-0.40325332  1.01327135 -0.71521823 ... -0.63717631  0.06529747
  -0.65159505]
 ...
 [-0.40547014  2.95931752 -1.30336132 ... -0.59225149  0.37901005
  -0.91069248]
 [ 0.85189733 -0.50032012  1.03323679 ...  0.84534281 -2.69458597
   1.52257036]
 [-0.38135592 -0.50032012 -0.35216694 ...  1.15981657 -3.12158061
  -0.25731635]]
[[-0.39680933 -0.50032012 -1.00711147 ... -0.77195078  0.42887165
  -0.48121032]
 [-0.40079621  1.2294987  -0.66439105 ... -0.32270256  0.44418004
  -1.25709448]
 [-0.39523443 -0.50032012  2.43316256 ...  0.80041799  0.36993437
   0.79033849]
 ...
 [ 1.1779666  -0.50032012  1.03323679 ...  0.84534281 -3.5124912
   1.23390203]
 [-0.40131065 -0.50032012 -0.94321444 ...  0.08162084  0.43422959
   0.05669848]
 [-0.39643252 -0.50032012  0.26792466 ...  0.17147048  0.29470458
  -0.49529171]]


In [4]:
from sklearn.svm import SVR  # For Support Vector Regression (SVM for regression)
from sklearn.metrics import mean_squared_error, r2_score

# Create an SVM model instance
svm_model = SVR(kernel='linear')  # You can choose different kernels (e.g., linear, RBF) based on your problem

# Fit the model to the training data
svm_model.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = svm_model.predict(X_test_scaled)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared (R2) Score:", r2)


Mean Squared Error: 28.91852267161823
R-squared (R2) Score: 0.6056589279132606


In [5]:
from sklearn.model_selection import GridSearchCV

# Define a parameter grid with the hyperparameters to search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'epsilon': [0.01, 0.1, 1]
}

# Create the SVM model instance
svm_model = SVR()

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the GridSearchCV to the training data
grid_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}


In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

def find_best_svm_model(X_train, y_train, X_test, y_test, param_grid, cv=5, scoring='neg_mean_squared_error'):
    # Create an SVM model instance
    svm_model = SVR()

    # Create a GridSearchCV object
    grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, scoring=scoring, cv=cv)

    # Fit the GridSearchCV to the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_

    # Get the best SVM model with the best hyperparameters
    best_svm_model = grid_search.best_estimator_

    # Make predictions on the test data with the best model
    y_pred = best_svm_model.predict(X_test)

    # Evaluate the best model's performance
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return best_svm_model, best_params, mse, r2

# Example usage:
best_hyperparameters = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'epsilon': [0.01, 0.1, 1]
}

best_svm_model, best_params, mse, r2 = find_best_svm_model(X_train_scaled, y_train, X_test_scaled, y_test, best_hyperparameters)

print("Best Hyperparameters:", best_params)
print("Mean Squared Error (Best Model):", mse)
print("R-squared (R2) Score (Best Model):", r2)


Best Hyperparameters: {'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}
Mean Squared Error (Best Model): 12.7381920196588
R-squared (R2) Score (Best Model): 0.8262984470361981


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

def find_best_random_forest_model(X_train, y_train, X_test, y_test, param_grid, cv=5, scoring='neg_mean_squared_error'):
    # Create a Random Forest model instance
    rf_model = RandomForestRegressor()

    # Create a GridSearchCV object
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring=scoring, cv=cv)

    # Fit the GridSearchCV to the training data
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_

    # Get the best Random Forest model with the best hyperparameters
    best_rf_model = grid_search.best_estimator_

    # Make predictions on the test data with the best model
    y_pred = best_rf_model.predict(X_test)

    # Evaluate the best model's performance
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return best_rf_model, best_params, mse, r2


In [9]:
# Assuming you have already defined X_train_scaled, y_train, X_test_scaled, and y_test

# Define hyperparameters for SVM and Random Forest
best_svm_hyperparameters = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'epsilon': [0.01, 0.1, 1]
}

best_rf_hyperparameters = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Find the best SVM and Random Forest models
best_svm_model, best_svm_params, svm_mse, svm_r2 = find_best_svm_model(X_train_scaled, y_train, X_test_scaled, y_test, best_svm_hyperparameters)
best_rf_model, best_rf_params, rf_mse, rf_r2 = find_best_random_forest_model(X_train_scaled, y_train, X_test_scaled, y_test, best_rf_hyperparameters)

# Print the results
print("Best Hyperparameters (SVM):", best_svm_params)
print("Mean Squared Error (Best SVM Model):", svm_mse)
print("R-squared (R2) Score (Best SVM Model):", svm_r2)
print("\n")
print("Best Hyperparameters (Random Forest):", best_rf_params)
print("Mean Squared Error (Best Random Forest Model):", rf_mse)
print("R-squared (R2) Score (Best Random Forest Model):", rf_r2)


Best Hyperparameters (SVM): {'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}
Mean Squared Error (Best SVM Model): 12.7381920196588
R-squared (R2) Score (Best SVM Model): 0.8262984470361981


Best Hyperparameters (Random Forest): {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
Mean Squared Error (Best Random Forest Model): 9.255978586985396
R-squared (R2) Score (Best Random Forest Model): 0.8737828843938149


In [10]:
#the random forest algorithm is better because of a better R_squared and a lower mean squared error

# Exercice 2 

In [1]:
import seaborn as sns

# Load the Titanic dataset
titanic_dataset = sns.load_dataset('titanic')

# Display the first few rows of the dataset
print(titanic_dataset.head())


   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the Titanic dataset
titanic_dataset = sns.load_dataset('titanic')

# Handle missing values (you may need a more sophisticated approach)
titanic_dataset = titanic_dataset.dropna(subset=['age', 'embarked'])


# Select features and target variable
df_encoded = pd.get_dummies(X['pclass'], prefix='pclass')
X = titanic_dataset[['sex', 'age', 'fare']]
X = pd.concat([X, df_encoded], axis=1)
y = titanic_dataset['survived']

# Convert categorical variables to numerical
le = LabelEncoder()
X['sex'] = le.fit_transform(X['sex'])


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [8]:
# Dictionary to store accuracies
accuracies = {}

for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)

    # Store the accuracy in the dictionary
    accuracies[model_name] = accuracy

# Print the accuracies
for model_name, accuracy in accuracies.items():
    print(f"{model_name}: {accuracy:.2f}")


Random Forest: 0.78
Support Vector Machine: 0.64
K-Nearest Neighbors: 0.66


C:\Users\louis\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\louis\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\louis\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\louis\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version